# Prophet Predictions for the Bremen Big Data Challenge

## Importing the necessary libraries and loading the dataset

In [1]:
import pandas as pd
from prophet import Prophet
import warnings
warnings.filterwarnings("ignore") # ignore the warnings in the predictions

In [2]:
df = pd.read_csv('df_cleaned_both_directions_interpolated.csv')
df['Datum'] = pd.to_datetime(df['Datum'])  

## Predicting the best values for Temperatur & Salinität

### Temperatur 2004

In [3]:
%%time
df_prophet_past=df[:15340] #specifying the data (until 2004)
df_prophet_past=df_prophet_past[['Datum','Temperatur']]
df_prophet_past.columns = ['ds','y']
model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_range= 0.9 
        , changepoint_prior_scale=0.5
        , weekly_seasonality=True
        , seasonality_prior_scale= 1
        )
    
    # Fit the model to the data
model.fit(df_prophet_past)
    

    # Make predictions for the future
future = model.make_future_dataframe(periods=365, freq='D', include_history=False)
future['floor'] = 0
forecast = model.predict(future)

predicted_Temperatur_past1 = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_Temperatur'})

Wall time: 1min 25s


In [4]:
predicted_Temperatur_past1.mean()

predicted_Temperatur    10.945293
dtype: float64

### Salinität 2004

In [5]:
%%time
df_prophet_past=df[:15340] #specifying the data (until 2004)
df_prophet_past=df_prophet_past[['Datum','Salinität']]
df_prophet_past.columns = ['ds','y']
model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_range= 0.8 
        , changepoint_prior_scale=1
        , weekly_seasonality=True
        , yearly_seasonality=True
        , seasonality_prior_scale= 10
        )

model.fit(df_prophet_past)

future = model.make_future_dataframe(periods=365, freq='D', include_history=False)
future['floor'] = 0
forecast = model.predict(future)

predicted_Salinität_past1 = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_Salinität'})

Wall time: 50.1 s


In [6]:
predicted_Salinität_past1.mean()

predicted_Salinität    32.191829
dtype: float64

### Inlcuding the predictions in the training set for 2011 - 2013

In [7]:
past_dataframe= pd.concat(
    [
        predicted_Salinität_past1.iloc[:, 0], # ds column
         predicted_Temperatur_past1.iloc[:, 1],
         predicted_Salinität_past1.iloc[:, 1]],
        axis=1)

past_dataframe

,ds,predicted_Temperatur,predicted_Salinität
0,2004-01-01,6.557466,32.792246
1,2004-01-02,6.468121,32.790360
2,2004-01-03,6.388836,32.776079
3,2004-01-04,6.311589,32.758008
4,2004-01-05,6.237441,32.741909
...,...,...,...
360,2004-12-26,7.098509,32.841153
361,2004-12-27,7.012904,32.821528
362,2004-12-28,6.908693,32.829348
363,2004-12-29,6.817693,32.832736


In [8]:
past_dataframe = past_dataframe.rename(
    columns={'ds': 'Datum'
            , 'predicted_Temperatur': 'Temperatur'
            ,'predicted_Salinität': 'Salinität'
            })
past_dataframe

,Datum,Temperatur,Salinität
0,2004-01-01,6.557466,32.792246
1,2004-01-02,6.468121,32.790360
2,2004-01-03,6.388836,32.776079
3,2004-01-04,6.311589,32.758008
4,2004-01-05,6.237441,32.741909
...,...,...,...
360,2004-12-26,7.098509,32.841153
361,2004-12-27,7.012904,32.821528
362,2004-12-28,6.908693,32.829348
363,2004-12-29,6.817693,32.832736


In [10]:
df = pd.read_csv('df_cleaned_both_directions_interpolated.csv')
df['Datum'] = pd.to_datetime(df['Datum'])  
df_pre_2004 =df[:15340]
df_pre_2004.tail()

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
0,1962-01-01,1.200000,4.6,33.480000,1.520000,7.280000,10.2
1,1962-01-02,1.200000,4.6,33.480000,1.520000,7.280000,10.2
2,1962-01-03,1.200000,5.1,33.480000,1.520000,7.280000,10.2
3,1962-01-04,1.200000,4.3,33.480000,1.520000,7.280000,10.2
4,1962-01-05,1.200000,5.1,33.480000,1.520000,7.280000,10.2
...,...,...,...,...,...,...,...
15335,2003-12-27,1.100000,7.0,31.331333,1.990000,7.320000,10.2
15336,2003-12-28,1.150000,7.0,31.342167,2.065000,7.855000,10.2
15337,2003-12-29,1.200000,7.0,31.353000,2.140000,8.390000,10.2
15338,2003-12-30,1.500000,6.9,31.425000,2.340000,5.610000,10.2


In [11]:
df2 = pd.concat([df_pre_2004, past_dataframe]) 
df2.shape

(15705, 7)

In [12]:
df2.tail()

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
360,2004-12-26,NaN,7.098509,32.841153,NaN,NaN,NaN
361,2004-12-27,NaN,7.012904,32.821528,NaN,NaN,NaN
362,2004-12-28,NaN,6.908693,32.829348,NaN,NaN,NaN
363,2004-12-29,NaN,6.817693,32.832736,NaN,NaN,NaN
364,2004-12-30,NaN,6.725795,32.809914,NaN,NaN,NaN


In [13]:
df_after_2004 = df[15340:]
df_after_2004

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
15340,2005-01-01,0.505405,6.900000,33.199357,4.398811,5.779081,10.200000
15341,2005-01-02,0.502703,6.900000,33.204178,4.404405,5.779541,10.200000
15342,2005-01-03,0.500000,6.900000,33.209000,4.410000,5.780000,10.200000
15343,2005-01-04,0.500000,5.500000,31.277000,3.260000,20.760000,24.020000
15344,2005-01-05,1.000000,6.200000,32.278000,3.550000,19.780000,23.320000
...,...,...,...,...,...,...,...
17526,2010-12-27,2.033333,3.116667,30.388000,0.748333,23.913333,24.663333
17527,2010-12-28,1.900000,2.900000,30.137000,0.810000,26.100000,26.910000
17528,2010-12-29,2.000000,2.100000,30.273000,0.790000,26.730000,27.520000
17529,2010-12-30,3.000000,3.600000,31.459000,0.640000,13.920000,14.550000


In [14]:
df_with_2004 = pd.concat([df, df_after_2004])
df_with_2004 = df_with_2004.reset_index(drop=True)
df_with_2004.tail()

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
19717,2010-12-27,2.033333,3.116667,30.388,0.748333,23.913333,24.663333
19718,2010-12-28,1.900000,2.900000,30.137,0.810000,26.100000,26.910000
19719,2010-12-29,2.000000,2.100000,30.273,0.790000,26.730000,27.520000
19720,2010-12-30,3.000000,3.600000,31.459,0.640000,13.920000,14.550000
19721,2010-12-31,3.000000,3.600000,31.459,0.640000,13.920000,14.550000


## Temperatur 2011 - 2013

In [15]:
%%time
df = df_with_2004[['Datum', 'SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','SECCI','y', 'Sal', 'NO2', 'NO3']

model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_prior_scale=0.002
        , holidays_prior_scale=10)
model.add_country_holidays(country_name='Germany')

model.fit(df)

future = model.make_future_dataframe(periods=1095, freq='D', include_history=False)

forecast = model.predict(future)

predicted_df_Temperatur1 = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'Temperatur'})

Wall time: 18.9 s


## Salinität 2011 - 2013

In [16]:
%%time
df = df_with_2004[['Datum', 'SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','SECCI','Temperatur', 'y', 'NO2', 'NO3']

model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_prior_scale=0.5
        , weekly_seasonality=True
        , yearly_seasonality=True
        , seasonality_prior_scale= 0.4
        , changepoint_range= 0.8)

model.fit(df)

future = model.make_future_dataframe(periods=1095, freq='D', include_history=False)

forecast = model.predict(future)

predicted_df_Salinität1 = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'Salinität'})

Wall time: 1min 24s


## Predicting the best values for the other variables

Note: Since the best values for the other variables come from a previous version, which included using the variables Temperatur as well as Salinität as additional regressors, we predict those values seperately and merge them with the best values later on.

# Predicting the best values for SECCI, NO2, NO3 and NOX

## Loading the dataset 

In [17]:
# load and prepare dataframe
df = pd.read_csv('df_cleaned_both_directions_interpolated.csv')
df['Datum'] = pd.to_datetime(df['Datum'])  
df = df[['Datum','SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','SECCI','Temperatur','Sal', 'y', 'NO3']
df = df[:15340] 

## NO2 2004 

In [18]:
%%time
model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_range= 0.85 
        , changepoint_prior_scale=0.5
        , holidays_prior_scale=10
        , weekly_seasonality=True
        , yearly_seasonality=10
        , seasonality_prior_scale= 1,
         seasonality_mode='additive'
        )
model.add_country_holidays(country_name='Germany') 
    
    # Fit the model to the data
model.fit(df)
    

    # Make predictions for the future
future = model.make_future_dataframe(periods=365, freq='D', include_history=False)
forecast = model.predict(future)

    # Extract the predicted values and return as a DataFrame
predicted_df_past_NO2 = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_No2'})

Wall time: 1min 23s


In [19]:
predicted_df_past_NO2

,ds,predicted_No2
0,2004-01-01,1.494183
1,2004-01-02,1.425386
2,2004-01-03,1.424524
3,2004-01-04,1.423335
4,2004-01-05,1.421703
...,...,...
360,2004-12-26,1.400213
361,2004-12-27,1.320602
362,2004-12-28,1.318597
363,2004-12-29,1.324878


In [20]:
predicted_df_past_NO2.mean()

predicted_No2    0.553223
dtype: float64

## Temperatur 2004 (using as an additional regressor for other variables)

In [21]:
# load and prepare dataframe
df = pd.read_csv('df_cleaned_both_directions_interpolated.csv')
df['Datum'] = pd.to_datetime(df['Datum'])  
df = df[['Datum','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','y','Salinität', 'NO2', 'NO3']
df = df[:15340]

In [22]:
%%time
model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_range= 0.9 
        , changepoint_prior_scale=0.5
        , weekly_seasonality=True
        , yearly_seasonality=True
        , seasonality_prior_scale= 0.01
        , seasonality_mode='multiplicative'
        )

    # Fit the model to the data
model.fit(df)
    

    # Make predictions for the future
future = model.make_future_dataframe(periods=365, freq='D', include_history=False)
forecast = model.predict(future)

    # Extract the predicted values and return as a DataFrame
predicted_df_past_Temperatur = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_Temperatur'})

Wall time: 1min 26s


In [23]:
predicted_df_past_Temperatur

,ds,predicted_Temperatur
0,2004-01-01,6.087548
1,2004-01-02,5.990730
2,2004-01-03,5.905549
3,2004-01-04,5.822614
4,2004-01-05,5.743099
...,...,...
360,2004-12-26,6.670542
361,2004-12-27,6.577482
362,2004-12-28,6.462872
363,2004-12-29,6.363249


In [24]:
predicted_df_past_Temperatur.mean()

predicted_Temperatur    10.891762
dtype: float64

## Salinität 2004  (using as additional regressor for other variables)

In [25]:
# load and prepare dataframe
df = pd.read_csv('df_cleaned_both_directions_interpolated.csv')
df['Datum'] = pd.to_datetime(df['Datum'])  
df = df[['Datum','SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','SECCI','Temperatur','y', 'NO2', 'NO3']
df = df[:15340]

In [26]:
%%time
model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_range= 0.8 
        , changepoint_prior_scale=1
        , weekly_seasonality=True
        , yearly_seasonality=True
        , seasonality_prior_scale= 10
        )

    # Fit the model to the data
model.fit(df)
    

    # Make predictions for the future
future = model.make_future_dataframe(periods=365, freq='D', include_history=False)

forecast = model.predict(future)

    # Extract the predicted values and return as a DataFrame
predicted_df_past_Salinität = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_Salinität'})

Wall time: 50.7 s


In [27]:
predicted_df_past_Salinität

,ds,predicted_Salinität
0,2004-01-01,32.792246
1,2004-01-02,32.790360
2,2004-01-03,32.776079
3,2004-01-04,32.758008
4,2004-01-05,32.741909
...,...,...
360,2004-12-26,32.841153
361,2004-12-27,32.821528
362,2004-12-28,32.829348
363,2004-12-29,32.832736


In [28]:
predicted_df_past_Salinität.mean()

predicted_Salinität    32.191829
dtype: float64

## SECCI 2004 

In [29]:
# load and prepare dataframe
df = pd.read_csv('df_cleaned_both_directions_interpolated.csv')
df['Datum'] = pd.to_datetime(df['Datum'])  
df = df[['Datum','SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','y','Temperatur','Salinität', 'NO2', 'NO3']
df = df[:15340]

In [30]:
%%time
model = Prophet(interval_width=0.95
        , daily_seasonality=25
        , changepoint_range= 0.8
        , changepoint_prior_scale=0.5
        , holidays_prior_scale=1
        , weekly_seasonality=10
        , yearly_seasonality=10
        , seasonality_prior_scale= 7.5
         ,seasonality_mode='multiplicative'
        )
model.add_country_holidays(country_name='Germany') 
model.add_regressor('Salinität')

    # Fit the model to the data
model.fit(df)
    

    # Make predictions for the future
future = model.make_future_dataframe(periods=365, freq='D', include_history=False)
future['Salinität'] =  predicted_df_past_Salinität['predicted_Salinität'].values

forecast = model.predict(future)

    # Extract the predicted values and return as a DataFrame
predicted_df_past_SECCI = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_SECCI'})

Wall time: 1min 35s


In [31]:
predicted_df_past_SECCI

,ds,predicted_SECCI
0,2004-01-01,2.610326
1,2004-01-02,2.450497
2,2004-01-03,2.488198
3,2004-01-04,2.525387
4,2004-01-05,2.567705
...,...,...
360,2004-12-26,2.458172
361,2004-12-27,2.419668
362,2004-12-28,2.385495
363,2004-12-29,2.426905


In [32]:
predicted_df_past_SECCI.mean()

predicted_SECCI    3.819767
dtype: float64

## NO3 2004

In [33]:
# load and prepare dataframe
df = pd.read_csv('df_cleaned_both_directions_interpolated.csv')
df['Datum'] = pd.to_datetime(df['Datum'])  
df = df[['Datum','SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','SECCI','Temperatur','Salinität', 'NO2', 'y']
df = df[:15340]

In [34]:
%%time
model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_range= 0.8 
        , changepoint_prior_scale=0.4
        , holidays_prior_scale=0.01
        , weekly_seasonality=True 
        , yearly_seasonality=True
        , seasonality_prior_scale= 5
        # seasonality_mode='additive'
        )
model.add_country_holidays(country_name='Germany') 
    
    # Fit the model to the data
model.fit(df)
    

    # Make predictions for the future
future = model.make_future_dataframe(periods=365, freq='D', include_history=False)

forecast = model.predict(future)

    # Extract the predicted values and return as a DataFrame
predicted_df_past_NO3 = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_No3'})

Wall time: 1min 40s


In [35]:
predicted_df_past_NO3

,ds,predicted_No3
0,2004-01-01,10.434577
1,2004-01-02,11.315191
2,2004-01-03,11.714399
3,2004-01-04,12.110239
4,2004-01-05,12.499488
...,...,...
360,2004-12-26,8.971164
361,2004-12-27,9.588421
362,2004-12-28,9.603213
363,2004-12-29,9.617251


In [36]:
predicted_df_past_NO3.mean()

predicted_No3    9.431756
dtype: float64

### Deletion of negative values in the NO3 column (2004)

In [37]:
print('mean before deletion',predicted_df_past_NO3.mean())
predicted_df_past_NO3["predicted_No3"][predicted_df_past_NO3["predicted_No3"] < 0] = 0
print('mean after deletion',predicted_df_past_NO3.mean())

mean before deletion predicted_No3    9.431756
dtype: float64
mean after deletion predicted_No3    9.521382
dtype: float64


## Generating NOX 2004

In [38]:
predicted_df_past_NOX = pd.DataFrame(columns = ['ds','NOX'])

predicted_df_past_NOX["ds"] = predicted_df_past_NO2['ds']
predicted_df_past_NOX['NOX'] = predicted_df_past_NO2['predicted_No2'] + predicted_df_past_NO3['predicted_No3']

predicted_df_past_NOX

,ds,NOX
0,2004-01-01,11.928760
1,2004-01-02,12.740577
2,2004-01-03,13.138922
3,2004-01-04,13.533574
4,2004-01-05,13.921191
...,...,...
360,2004-12-26,10.371377
361,2004-12-27,10.909024
362,2004-12-28,10.921811
363,2004-12-29,10.942129


## Concating the dataframe and renaming the columns (2004)

In [39]:
predicted_df_past = pd.concat(
    [
        predicted_df_past_SECCI.iloc[:, 0], # ds column
        predicted_df_past_SECCI.iloc[:, 1], 
        predicted_df_past_Temperatur.iloc[:, 1],
        predicted_df_past_Salinität.iloc[:, 1], 
        predicted_df_past_NO2.iloc[:, 1],
        predicted_df_past_NO3.iloc[:, 1],
        predicted_df_past_NOX.iloc[:,1]],
        axis=1)

predicted_df_past

,ds,predicted_SECCI,predicted_Temperatur,predicted_Salinität,predicted_No2,predicted_No3,NOX
0,2004-01-01,2.610326,6.087548,32.792246,1.494183,10.434577,11.928760
1,2004-01-02,2.450497,5.990730,32.790360,1.425386,11.315191,12.740577
2,2004-01-03,2.488198,5.905549,32.776079,1.424524,11.714399,13.138922
3,2004-01-04,2.525387,5.822614,32.758008,1.423335,12.110239,13.533574
4,2004-01-05,2.567705,5.743099,32.741909,1.421703,12.499488,13.921191
...,...,...,...,...,...,...,...
360,2004-12-26,2.458172,6.670542,32.841153,1.400213,8.971164,10.371377
361,2004-12-27,2.419668,6.577482,32.821528,1.320602,9.588421,10.909024
362,2004-12-28,2.385495,6.462872,32.829348,1.318597,9.603213,10.921811
363,2004-12-29,2.426905,6.363249,32.832736,1.324878,9.617251,10.942129


In [40]:
predicted_df_past = predicted_df_past.rename(
    columns={'ds': 'Datum'
            , 'predicted_SECCI': 'SECCI' 
            , 'predicted_Temperatur': 'Temperatur'
            ,'predicted_Salinität': 'Salinität'
            ,'predicted_No2': 'NO2'
            ,'predicted_No3': 'NO3'
            ,'NOX': 'NOx'
            })
predicted_df_past

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
0,2004-01-01,2.610326,6.087548,32.792246,1.494183,10.434577,11.928760
1,2004-01-02,2.450497,5.990730,32.790360,1.425386,11.315191,12.740577
2,2004-01-03,2.488198,5.905549,32.776079,1.424524,11.714399,13.138922
3,2004-01-04,2.525387,5.822614,32.758008,1.423335,12.110239,13.533574
4,2004-01-05,2.567705,5.743099,32.741909,1.421703,12.499488,13.921191
...,...,...,...,...,...,...,...
360,2004-12-26,2.458172,6.670542,32.841153,1.400213,8.971164,10.371377
361,2004-12-27,2.419668,6.577482,32.821528,1.320602,9.588421,10.909024
362,2004-12-28,2.385495,6.462872,32.829348,1.318597,9.603213,10.921811
363,2004-12-29,2.426905,6.363249,32.832736,1.324878,9.617251,10.942129


## Including the predicted values of 2004 in the dataframe

In [76]:
df=pd.read_csv('df_cleaned_both_directions_interpolated.csv')
df_pre_2004=df[:15340]
df3=pd.concat([df_pre_2004, predicted_df_past])
df3

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
0,1962-01-01,1.200000,4.600000,33.480000,1.520000,7.280000,10.200000
1,1962-01-02,1.200000,4.600000,33.480000,1.520000,7.280000,10.200000
2,1962-01-03,1.200000,5.100000,33.480000,1.520000,7.280000,10.200000
3,1962-01-04,1.200000,4.300000,33.480000,1.520000,7.280000,10.200000
4,1962-01-05,1.200000,5.100000,33.480000,1.520000,7.280000,10.200000
...,...,...,...,...,...,...,...
360,2004-12-26 00:00:00,2.458172,7.098509,32.841153,1.400213,8.971164,10.371377
361,2004-12-27 00:00:00,2.419668,7.012904,32.821528,1.320602,9.588421,10.909024
362,2004-12-28 00:00:00,2.385495,6.908693,32.829348,1.318597,9.603213,10.921811
363,2004-12-29 00:00:00,2.426905,6.817693,32.832736,1.324878,9.617251,10.942129


In [77]:
df=pd.read_csv('df_cleaned_both_directions_interpolated.csv')
df_after_2004 = df[15340:]
df_after_2004

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
15340,2005-01-01,0.505405,6.900000,33.199357,4.398811,5.779081,10.200000
15341,2005-01-02,0.502703,6.900000,33.204178,4.404405,5.779541,10.200000
15342,2005-01-03,0.500000,6.900000,33.209000,4.410000,5.780000,10.200000
15343,2005-01-04,0.500000,5.500000,31.277000,3.260000,20.760000,24.020000
15344,2005-01-05,1.000000,6.200000,32.278000,3.550000,19.780000,23.320000
...,...,...,...,...,...,...,...
17526,2010-12-27,2.033333,3.116667,30.388000,0.748333,23.913333,24.663333
17527,2010-12-28,1.900000,2.900000,30.137000,0.810000,26.100000,26.910000
17528,2010-12-29,2.000000,2.100000,30.273000,0.790000,26.730000,27.520000
17529,2010-12-30,3.000000,3.600000,31.459000,0.640000,13.920000,14.550000


In [78]:
df_with_2004 = pd.concat([df3, df_after_2004])
df_with_2004 = df_with_2004.reset_index(drop=True)
df_with_2004.tail()

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
17891,2010-12-27,2.033333,3.116667,30.388,0.748333,23.913333,24.663333
17892,2010-12-28,1.900000,2.900000,30.137,0.810000,26.100000,26.910000
17893,2010-12-29,2.000000,2.100000,30.273,0.790000,26.730000,27.520000
17894,2010-12-30,3.000000,3.600000,31.459,0.640000,13.920000,14.550000
17895,2010-12-31,3.000000,3.600000,31.459,0.640000,13.920000,14.550000


## Predicting the values of SECCI, NO2, NO3 and NOX for 2011 - 2013

## NO2 2011 - 2013

In [79]:
# loading and preparing the dataset
df = df_with_2004[['Datum','SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','SECCI','Temperatur','Salinität', 'y', 'NO3']

In [80]:
%%time
model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_range= 0.8 
        , changepoint_prior_scale=0.002
        , holidays_prior_scale=1
        , weekly_seasonality=True
        , yearly_seasonality=75
        , seasonality_prior_scale= 10
        )
model.add_country_holidays(country_name='Germany') 
    
    # Fit the model to the data
model.fit(df)

    # Make predictions for the future
future = model.make_future_dataframe(periods=1095, freq='D', include_history=False)

forecast = model.predict(future)

    # Extract the predicted values and return as a DataFrame
predicted_df_future_NO2 = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_No2'})

Wall time: 20.4 s


In [81]:
predicted_df_future_NO2

,ds,predicted_No2
0,2011-01-01,1.651253
1,2011-01-02,1.460906
2,2011-01-03,1.512136
3,2011-01-04,1.520109
4,2011-01-05,1.506363
...,...,...
1090,2013-12-26,1.346184
1091,2013-12-27,1.333207
1092,2013-12-28,1.368846
1093,2013-12-29,1.386527


In [82]:
predicted_df_future_NO2.mean()

predicted_No2    0.564845
dtype: float64

# Temperatur 2011 - 2013

In [83]:
df = df_with_2004[['Datum','SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','SECCI','y','Salinität', 'NO2', 'NO3']

In [84]:
%%time
model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_range=0.8
        , changepoint_prior_scale=0.8
        , holidays_prior_scale=10
        , weekly_seasonality=True
        , yearly_seasonality=True
        ,seasonality_prior_scale= 10
        , seasonality_mode='multiplicative'
               )
model.add_country_holidays(country_name='Germany')
model.add_regressor('NO2')
    
    # Fit the model to the data
model.fit(df)
    

    # Make predictions for the future
future = model.make_future_dataframe(periods= 1095, freq='D', include_history=False)
future['NO2'] = predicted_df_future_NO2['predicted_No2'].values

forecast = model.predict(future)

    # Extract the predicted values and return as a DataFrame
predicted_df_future_Temperatur = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_Temperatur'})

Wall time: 1min 34s


In [85]:
predicted_df_future_Temperatur

,ds,predicted_Temperatur
0,2011-01-01,6.128479
1,2011-01-02,5.881766
2,2011-01-03,5.793545
3,2011-01-04,5.693905
4,2011-01-05,5.622595
...,...,...
1090,2013-12-26,6.327203
1091,2013-12-27,6.329220
1092,2013-12-28,6.244407
1093,2013-12-29,6.160130


In [86]:
predicted_df_future_Temperatur.mean()

predicted_Temperatur    10.491062
dtype: float64

## Salinität 2011 - 2013

In [87]:
df= df_with_2004[['Datum','SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','SECCI','Temperatur','y', 'NO2', 'NO3']

In [88]:
%%time
model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_range= 0.8 
        , changepoint_prior_scale=0.5
        , weekly_seasonality=True
        , yearly_seasonality=True
        , seasonality_prior_scale= 0.4)
    
model.add_regressor('Temperatur')
    # Fit the model to the data
model.fit(df)
    

    # Make predictions for the future
future = model.make_future_dataframe(periods=1095, freq='D', include_history=False)
future['Temperatur'] = predicted_df_future_Temperatur['predicted_Temperatur'].values

forecast = model.predict(future)

    # Extract the predicted values and return as a DataFrame
predicted_df_future_Salinität = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_Salinität'})

Wall time: 1min 34s


In [89]:
predicted_df_future_Salinität

,ds,predicted_Salinität
0,2011-01-01,33.021584
1,2011-01-02,33.026013
2,2011-01-03,33.014173
3,2011-01-04,33.028105
4,2011-01-05,33.038119
...,...,...
1090,2013-12-26,33.203275
1091,2013-12-27,33.193009
1092,2013-12-28,33.180594
1093,2013-12-29,33.163280


In [90]:
predicted_df_future_Salinität.mean()

predicted_Salinität    32.512906
dtype: float64

## SECCI 2011 - 2013

In [91]:
df = df_with_2004[['Datum','SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','y','Temperatur','Salinität', 'NO2', 'NO3']

In [92]:
%%time
model = Prophet(interval_width=0.95
        , daily_seasonality=25
        , changepoint_range= 0.9 
        , changepoint_prior_scale=0.5
        , holidays_prior_scale=1
        , weekly_seasonality=True
        , yearly_seasonality=True
        , seasonality_prior_scale= 10
        , seasonality_mode='multiplicative'
        )
model.add_country_holidays(country_name='Germany') 
model.add_regressor('Temperatur')
model.add_regressor('Salinität')
    
    # Fit the model to the data
model.fit(df)
    

    # Make predictions for the future
future = model.make_future_dataframe(periods=1095, freq='D', include_history=False)
future['Temperatur'] = predicted_df_future_Temperatur['predicted_Temperatur'].values
future['Salinität'] =  predicted_df_future_Salinität['predicted_Salinität'].values

forecast = model.predict(future)

    # Extract the predicted values and return as a DataFrame
predicted_df_future_SECCI = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_SECCI'})

Wall time: 1min 33s


In [93]:
predicted_df_future_SECCI

,ds,predicted_SECCI
0,2011-01-01,2.522770
1,2011-01-02,2.377191
2,2011-01-03,2.413454
3,2011-01-04,2.409106
4,2011-01-05,2.432963
...,...,...
1090,2013-12-26,2.453290
1091,2013-12-27,2.253213
1092,2013-12-28,2.270094
1093,2013-12-29,2.289452


In [94]:
predicted_df_future_SECCI.mean()

predicted_SECCI    3.590886
dtype: float64

## NO3 2011 - 2013

In [95]:
df = df_with_2004[['Datum','SECCI','Temperatur', 'Salinität', 'NO2', 'NO3']]
df.columns = ['ds','SECCI','Temperatur','Salinität', 'NO2', 'y']

In [96]:
%%time
model = Prophet(interval_width=0.95
        , daily_seasonality=True
        , changepoint_range= 0.85 
        , changepoint_prior_scale=0.5
        , holidays_prior_scale=10
        , weekly_seasonality=75
        , yearly_seasonality=True
        , seasonality_prior_scale= 1)
model.add_country_holidays(country_name='Germany') 
model.add_regressor('Temperatur')
model.add_regressor('NO2')
model.add_regressor('SECCI')
    
    # Fit the model to the data
model.fit(df)
    

    # Make predictions for the future
future = model.make_future_dataframe(periods=1095, freq='D', include_history=False)
future['Temperatur'] =  predicted_df_future_Temperatur['predicted_Temperatur'].values
future['NO2'] = predicted_df_future_NO2['predicted_No2'].values
future['SECCI'] =  predicted_df_future_SECCI['predicted_SECCI'].values

forecast = model.predict(future)

    # Extract the predicted values and return as a DataFrame
predicted_df_future_NO3 = forecast[['ds', 'yhat']].rename(
    columns={'yhat': 'predicted_No3'})

Wall time: 6min 23s


In [97]:
predicted_df_future_NO3

,ds,predicted_No3
0,2011-01-01,9.201490
1,2011-01-02,11.303934
2,2011-01-03,11.774404
3,2011-01-04,11.791331
4,2011-01-05,11.728289
...,...,...
1090,2013-12-26,8.595497
1091,2013-12-27,9.247648
1092,2013-12-28,9.679292
1093,2013-12-29,10.086362


In [98]:
predicted_df_future_NO3.mean()

predicted_No3    9.192923
dtype: float64

### Deletion of negative values in the NO3 column (2011 - 2013)

In [99]:
print('mean before deletion',predicted_df_future_NO3.mean())
predicted_df_future_NO3["predicted_No3"][predicted_df_future_NO3["predicted_No3"] < 0] = 0
print('mean after deletion',predicted_df_future_NO3.mean())

mean before deletion predicted_No3    9.192923
dtype: float64
mean after deletion predicted_No3    9.25578
dtype: float64


## Generating NOX 2011 - 2013 

In [100]:
predicted_df_future_NOX = pd.DataFrame(columns = ['ds','NOX'])

predicted_df_future_NOX["ds"] = predicted_df_future_NO2['ds']
predicted_df_future_NOX['NOX'] = predicted_df_future_NO2['predicted_No2'] + predicted_df_future_NO3['predicted_No3']

predicted_df_future_NOX

,ds,NOX
0,2011-01-01,10.852744
1,2011-01-02,12.764840
2,2011-01-03,13.286540
3,2011-01-04,13.311440
4,2011-01-05,13.234651
...,...,...
1090,2013-12-26,9.941681
1091,2013-12-27,10.580855
1092,2013-12-28,11.048138
1093,2013-12-29,11.472890


## Concating the dataframe and renaming the columns (2011 - 2013)

In [101]:
predicted_df_past = pd.concat(
    [
        predicted_df_past_SECCI.iloc[:, 0], # ds column
        predicted_df_past_SECCI.iloc[:, 1], 
        predicted_Temperatur_past1.iloc[:, 1], # from the first prediction
        predicted_Salinität_past1.iloc[:, 1], # from the first prediction
        predicted_df_past_NO2.iloc[:, 1],
        predicted_df_past_NO3.iloc[:, 1], 
        predicted_df_past_NOX.iloc[:,1]],
        axis=1)

predicted_df_past

,ds,predicted_SECCI,predicted_Temperatur,predicted_Salinität,predicted_No2,predicted_No3,NOX
0,2004-01-01,2.610326,6.557466,32.792246,1.494183,10.434577,11.928760
1,2004-01-02,2.450497,6.468121,32.790360,1.425386,11.315191,12.740577
2,2004-01-03,2.488198,6.388836,32.776079,1.424524,11.714399,13.138922
3,2004-01-04,2.525387,6.311589,32.758008,1.423335,12.110239,13.533574
4,2004-01-05,2.567705,6.237441,32.741909,1.421703,12.499488,13.921191
...,...,...,...,...,...,...,...
360,2004-12-26,2.458172,7.098509,32.841153,1.400213,8.971164,10.371377
361,2004-12-27,2.419668,7.012904,32.821528,1.320602,9.588421,10.909024
362,2004-12-28,2.385495,6.908693,32.829348,1.318597,9.603213,10.921811
363,2004-12-29,2.426905,6.817693,32.832736,1.324878,9.617251,10.942129


In [102]:
predicted_df_past = predicted_df_past.rename(
    columns={'ds': 'Datum'
            , 'predicted_SECCI': 'SECCI' 
            , 'predicted_Temperatur': 'Temperatur'
            ,'predicted_Salinität': 'Salinität'
            ,'predicted_No2': 'NO2'
            ,'predicted_No3': 'NO3'
            ,'NOX': 'NOx'
            })
predicted_df_past

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
0,2004-01-01,2.610326,6.557466,32.792246,1.494183,10.434577,11.928760
1,2004-01-02,2.450497,6.468121,32.790360,1.425386,11.315191,12.740577
2,2004-01-03,2.488198,6.388836,32.776079,1.424524,11.714399,13.138922
3,2004-01-04,2.525387,6.311589,32.758008,1.423335,12.110239,13.533574
4,2004-01-05,2.567705,6.237441,32.741909,1.421703,12.499488,13.921191
...,...,...,...,...,...,...,...
360,2004-12-26,2.458172,7.098509,32.841153,1.400213,8.971164,10.371377
361,2004-12-27,2.419668,7.012904,32.821528,1.320602,9.588421,10.909024
362,2004-12-28,2.385495,6.908693,32.829348,1.318597,9.603213,10.921811
363,2004-12-29,2.426905,6.817693,32.832736,1.324878,9.617251,10.942129


In [103]:
predicted_df_future = pd.concat(
    [
        predicted_df_future_SECCI.iloc[:, 0], # ds column
        predicted_df_future_SECCI.iloc[:, 1], 
        predicted_df_Temperatur1.iloc[:, 1],# from the first prediction
        predicted_df_Salinität1.iloc[:, 1],# from the first prediction
        predicted_df_future_NO2.iloc[:, 1],
        predicted_df_future_NO3.iloc[:, 1], 
        predicted_df_future_NOX.iloc[:,1]],
        axis=1)

predicted_df_future

,ds,predicted_SECCI,Temperatur,Salinität,predicted_No2,predicted_No3,NOX
0,2011-01-01,2.522770,6.172535,32.801329,1.651253,9.201490,10.852744
1,2011-01-02,2.377191,6.136774,32.788713,1.460906,11.303934,12.764840
2,2011-01-03,2.413454,6.066448,32.777915,1.512136,11.774404,13.286540
3,2011-01-04,2.409106,5.969493,32.792802,1.520109,11.791331,13.311440
4,2011-01-05,2.432963,5.899283,32.804660,1.506363,11.728289,13.234651
...,...,...,...,...,...,...,...
1090,2013-12-26,2.453290,6.154982,32.742055,1.346184,8.595497,9.941681
1091,2013-12-27,2.253213,6.446216,32.744231,1.333207,9.247648,10.580855
1092,2013-12-28,2.270094,6.367648,32.733232,1.368846,9.679292,11.048138
1093,2013-12-29,2.289452,6.287543,32.717738,1.386527,10.086362,11.472890


In [104]:
predicted_df_future = predicted_df_future.rename(
    columns={'ds': 'Datum'
            , 'predicted_SECCI': 'SECCI' 
            , 'predicted_Temperatur': 'Temperatur'
            ,'predicted_Salinität': 'Salinität'
            ,'predicted_No2': 'NO2'
            ,'predicted_No3': 'NO3'
            ,'NOX': 'NOx'
            })
predicted_df_future

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
0,2011-01-01,2.522770,6.172535,32.801329,1.651253,9.201490,10.852744
1,2011-01-02,2.377191,6.136774,32.788713,1.460906,11.303934,12.764840
2,2011-01-03,2.413454,6.066448,32.777915,1.512136,11.774404,13.286540
3,2011-01-04,2.409106,5.969493,32.792802,1.520109,11.791331,13.311440
4,2011-01-05,2.432963,5.899283,32.804660,1.506363,11.728289,13.234651
...,...,...,...,...,...,...,...
1090,2013-12-26,2.453290,6.154982,32.742055,1.346184,8.595497,9.941681
1091,2013-12-27,2.253213,6.446216,32.744231,1.333207,9.247648,10.580855
1092,2013-12-28,2.270094,6.367648,32.733232,1.368846,9.679292,11.048138
1093,2013-12-29,2.289452,6.287543,32.717738,1.386527,10.086362,11.472890


## Saving the best predictions in a dataframe (2004 + 2011 - 2013)

In [105]:
predicted_df = pd.concat([predicted_df_past,predicted_df_future])
predicted_df["Datum"] = pd.to_datetime(predicted_df["Datum"], dayfirst=True)
predicted_df = predicted_df.reset_index(drop=True)
predicted_df

,Datum,SECCI,Temperatur,Salinität,NO2,NO3,NOx
0,2004-01-01,2.610326,6.557466,32.792246,1.494183,10.434577,11.928760
1,2004-01-02,2.450497,6.468121,32.790360,1.425386,11.315191,12.740577
2,2004-01-03,2.488198,6.388836,32.776079,1.424524,11.714399,13.138922
3,2004-01-04,2.525387,6.311589,32.758008,1.423335,12.110239,13.533574
4,2004-01-05,2.567705,6.237441,32.741909,1.421703,12.499488,13.921191
...,...,...,...,...,...,...,...
1455,2013-12-26,2.453290,6.154982,32.742055,1.346184,8.595497,9.941681
1456,2013-12-27,2.253213,6.446216,32.744231,1.333207,9.247648,10.580855
1457,2013-12-28,2.270094,6.367648,32.733232,1.368846,9.679292,11.048138
1458,2013-12-29,2.289452,6.287543,32.717738,1.386527,10.086362,11.472890


## Reading in the evaluation dataset

In [106]:
evaluation_df = pd.read_csv(r'bbdc_2023_AWI_data_evaluate_skeleton_student.csv', sep=';')
evaluation_df["Datum"] = pd.to_datetime(evaluation_df["Datum"], dayfirst=True)
evaluation_df = evaluation_df.set_index("Datum")
evaluation_df

,Uhrzeit,SECCI,Temperatur,Salinität,NO2,NO3,NOx
Datum,,,,,,,
NaT,NaN,Meter,°C,NaN,µmol/l,µmol/l,µmol/l
2004-01-02,7:34,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-05,14:10,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-06,9:05,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-07,8:30,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2013-12-18,7:52,NaN,NaN,NaN,NaN,NaN,NaN
2013-12-19,9:09,NaN,NaN,NaN,NaN,NaN,NaN
2013-12-20,8:00,NaN,NaN,NaN,NaN,NaN,NaN


## Merging the predictions with the evaluation dataset

In [107]:
merged_df = pd.merge(evaluation_df, predicted_df, on="Datum")
merged_df.drop(["SECCI_x", "Temperatur_x" , "Salinität_x","NO2_x", "NO3_x" ,"NOx_x"], axis = 1, inplace =True)
merged_df

,Datum,Uhrzeit,SECCI_y,Temperatur_y,Salinität_y,NO2_y,NO3_y,NOx_y
0,2004-01-02,7:34,2.450497,6.468121,32.790360,1.425386,11.315191,12.740577
1,2004-01-05,14:10,2.567705,6.237441,32.741909,1.421703,12.499488,13.921191
2,2004-01-06,9:05,2.548209,6.146821,32.754861,1.408748,12.464218,13.872966
3,2004-01-07,8:30,2.601930,6.071522,32.764908,1.404238,12.421925,13.826163
4,2004-01-08,9:00,2.609747,5.997364,32.750147,1.393146,12.295015,13.688161
...,...,...,...,...,...,...,...,...
909,2013-12-18,7:52,2.353983,7.266907,32.786598,1.208713,7.288957,8.497670
910,2013-12-19,9:09,2.338262,7.170536,32.775130,1.226904,7.244844,8.471748
911,2013-12-20,8:00,2.299456,7.080840,32.780276,1.242429,7.633278,8.875707
912,2013-12-23,8:10,2.305720,6.827347,32.742614,1.345377,8.939377,10.284754


In [108]:
merged_df.mean()

SECCI_y          3.679157
Temperatur_y    10.734964
Salinität_y     32.200563
NO2_y            0.547589
NO3_y            9.229150
NOx_y            9.776739
dtype: float64

In [109]:
merged_df.var()

SECCI_y          0.961928
Temperatur_y    24.041018
Salinität_y      0.214069
NO2_y            0.170243
NO3_y           66.825688
NOx_y           70.063183
dtype: float64

In [110]:
# saving the dataset as a .csv
import os 
os.makedirs('Prophet', exist_ok=True)  
merged_df.to_csv('BBDC\final_submission.csv', index=True) 